In [90]:
import pandas as pd
from src.api_osmr import get_matrix
from src.routing import optimize_routes
import numpy as np
import osmnx as ox
import folium  #0.14.0
import networkx as nx
from folium import plugins

In [91]:
def parse_coordinates(df):
    
    cities_list = cities.city
    coordinates = df[["lat", "lng"]]
    coordinates = list(coordinates.itertuples(index=False, name=None))
    coordinates = [f"{c[1]},{c[0]}" for c in coordinates]
    coordinates = ';'.join(coordinates)
    
    return coordinates

    

In [108]:
def plot_route(coordinates, dist=1000):
    
    # Calculate the median tuple
    median_tuple = tuple(np.median(np.array(coordinates), axis=0))

    # Calculate the Euclidean distance of each tuple from the median tuple
    distances = [np.linalg.norm(np.array(t) - np.array(median_tuple)) for t in coordinates]

    # Find the index of the tuple with the minimum distance
    closest_index = np.argmin(distances)

    # Get the tuple with the closest values to the median tuple
    closest_tuple = coordinates[closest_index]
    
    # graph = ox.graph_from_point(
    #     # (start_lat, start_lon),
    #     closest_tuple,
    #     dist=95000,
    #     # distance_type="bbox",
    #     network_type="drive",
    #     truncate_by_edge=False,
    #     simplify=True,
    #     retain_all=True,
    #     custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|residential|unclassified|service|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link"]',
    #     # infrastructure='way["highway"]',
    #     # clean_periphery=False,
    # )
    
    graph = ox.graph_from_point(center_point=closest_tuple, dist=dist, network_type='drive')
    
    map_center = coordinates[0]
    mymap = folium.Map(location=map_center, zoom_start=15, tiles="cartodbpositron")
    
    num_stops =len(coordinates)
    for i in range(num_stops-1):
        start_node = ox.distance.nearest_nodes(graph , coordinates[i][1],coordinates[i][0])
        end_node = ox.distance.nearest_nodes(graph , coordinates[i+1][1],coordinates[i+1][0])
        route = nx.shortest_path(
            graph,
            start_node,
            end_node,
            weight="length"
            )
        route_coordinates = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
        
        route_polyline = folium.PolyLine(locations=route_coordinates, color='red')
        mymap.add_child(route_polyline)
        
        ant_path = plugins.AntPath(locations=route_coordinates, color='blue', dash_array=[10, 50], delay=500, weight=5)
        mymap.add_child(ant_path)
    
    folium.Marker(location=coordinates[0], icon=folium.Icon(color='green', icon='play')).add_to(mymap)
    folium.Marker(location=coordinates[-1], icon=folium.Icon(color='red', icon='stop')).add_to(mymap)
    for coordinate in coordinates[1:-1]:
        folium.Marker(location=coordinate, icon=folium.Icon(color='blue', icon='store')).add_to(mymap)
    
    
    
    return mymap
    

In [93]:
places = pd.read_csv("data/input/pos.csv")
places.head()

,city,lat,lng,name,province,street
0,Alcanar,40.593153,0.561053,Bar Montsia Mar,Tarragona,"Carrer Tarongina, 2"
1,Alcanar,40.542112,0.481537,Bar Paella,Tarragona,"Carrer del Canonge Matamoros, 7"
2,Alcanar,40.543882,0.477914,Restaurant l’Esquella D’en Gaudí,Tarragona,"Carrer de l'Arquitecte Gaudí, 26"
3,Alcanar,40.528650,0.509533,Citrus del Tancat,Tarragona,C N-340 Km 1059 (Sol de Riu
4,Alcanar,40.569059,0.538400,Bar km.1065,Tarragona,"Carrer a Platja Daurada, 2"


In [94]:
cities = places.groupby(["city"])[["lat", "lng"]].median().reset_index()
cities.head()

,city,lat,lng
0,Alcanar,40.543398,0.481130
1,Amposta,40.708700,0.579551
2,Deltebre,40.719954,0.734037
3,L'Aldea,40.750078,0.622673
4,La Ràpita,40.618336,0.591224


In [95]:
cities_list = cities.city
coordinates = parse_coordinates(cities)
response_distances = get_matrix(coordinates)

# ortools only works with integers 
distances = np.array(response_distances["distances"]).astype(int)

In [96]:
# plot distances

In [97]:
starting_city = "Ulldecona"
depot_index = int(cities_list.index[cities_list==starting_city][0])
idx_route = optimize_routes(distances, depot_index, list(cities_list))

Route for vehicle:
 Ulldecona -> Alcanar -> Les Cases d'Alcanar -> La Ràpita -> Amposta -> Deltebre -> L'Aldea -> Tortosa -> Ulldecona
Route distance: 107.8km



In [98]:
df_best_route = cities.iloc[idx_route]
coordinates = list(zip(df_best_route['lat'], df_best_route['lng']))
plot_route(coordinates,dist=30000)


# Routing for each city

In [111]:
for city in places.city.unique():
    print(f"Planning route for {city}")
    df_city = places[places.city==city][:10].reset_index(drop=True)

    coordinates = parse_coordinates(df_city)
    response_distances = get_matrix(coordinates)

    distances = np.array(response_distances["distances"]).astype(int)
    pos_names = list(df_city.name)
    
    # Setting the farthest point as start
    # Setting the farthest point as start
    # Calculate the median tuple
    coordinates_tupples = np.array(response_distances["distances"]).astype(int)
    median_tuple = tuple(np.median(np.array(coordinates_tupples), axis=0))
    # Calculate the Euclidean distance of each tuple from the median tuple
    distances_tuples = [np.linalg.norm(np.array(t) - np.array(median_tuple)) for t in coordinates_tupples]
    # Find the index of the tuple with the maximum distance
    farthest_index = max(enumerate(distances_tuples), key=lambda x: x[1])[0]
    
    
    best_route = optimize_routes(distances, farthest_index, pos_names)[:-1]
    df_city.name[best_route].to_csv(f"data/output/{city}_routing.csv")

Planning route for Alcanar
Route for vehicle:
 Restaurante Club de Tenis Serramar -> Bar Montsia Mar -> Bar km.1065 -> Restaurante L’Estona -> Juan Pedro Lara Márquez -> Restaurant l’Esquella D’en Gaudí -> Bar Paella -> Sofía Alonso Beltrán -> Bar Moreno, C.B -> Citrus del Tancat -> Restaurante Club de Tenis Serramar
Route distance: 27.7km

Planning route for Amposta
Route for vehicle:
 Bar Residència L’Ullal Apasa -> Bar Restaurant Bon Gust -> Basar Amposta -> Prince Doner Kebab | Soylent Green -> Les Palmeres -> Bar Simpatica -> Amore Pizza - Doner Kebab | Soylent Green -> Únic Restaurant | Imperio Suichy -> Bar Fidel -> Bar Restaurant La Nova Torreta -> Bar Residència L’Ullal Apasa
Route distance: 4.7km

Planning route for Deltebre
Route for vehicle:
 Cafetería Bar Restaurante Lo Mirador -> Restaurant l'Arròs -> Restaurant Lo Transbordador -> Ristorante La Dolce Vita -> Cafeteria Restaurant Casa Nicanor -> Bar Restaurant de tots edades N40 -> Carrer Lovas Companies -> Restaurante @ 

In [112]:
city = "Ulldecona"
df_city = places[places.city==city][:10].reset_index(drop=True)


coordinates = parse_coordinates(df_city)
response_distances = get_matrix(coordinates)

distances = np.array(response_distances["distances"]).astype(int)
pos_names = list(df_city.name)

# Setting the farthest point as start
# Calculate the median tuple
coordinates_tupples = np.array(response_distances["distances"]).astype(int)
median_tuple = tuple(np.median(np.array(coordinates_tupples), axis=0))
# Calculate the Euclidean distance of each tuple from the median tuple
distances_tuples = [np.linalg.norm(np.array(t) - np.array(median_tuple)) for t in coordinates_tupples]
# Find the index of the tuple with the maximum distance
farthest_index = max(enumerate(distances_tuples), key=lambda x: x[1])[0]


best_route = optimize_routes(distances, farthest_index, pos_names)[:-1]

df_best_route = df_city.iloc[best_route]
coordinates = list(zip(df_best_route['lat'], df_best_route['lng']))
plot_route(coordinates, dist=2000)


Route for vehicle:
 Perpebrots_ulldecona -> El Rebost -> Bar lo negre -> Cafetría-Bar Phoenix -> Mont-Mar -> El Mosset d'Exequiel -> Kebab Ulldecona -> Bar La Plaça -> La Pollera -> Lo Paseo -> Perpebrots_ulldecona
Route distance: 5.0km

